In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)
print(keras.__version__)

1.13.1
2.2.4-tf


In [0]:
# model 생성
def create_model():
  # layer를 stack과 비슷하게 순차적으로 쌓는다.
  model = keras.Sequential()
  # 첫 layer의 input shape는 반드시 써줘야 한다. 그 다음 layer부터는 입력해주지 않아도 된다.
  model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu, padding='SAME', 
                                input_shape=(28, 28, 1)))
  model.add(keras.layers.MaxPool2D(padding='SAME'))
  model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
  model.add(keras.layers.MaxPool2D(padding='SAME'))
  model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
  model.add(keras.layers.MaxPool2D(padding='SAME'))
  # fully connected layer
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(256, activation=tf.nn.relu))
  model.add(keras.layers.Dropout(0.4))
  model.add(keras.layers.Dense(10))
  return model

In [0]:
# loss function
'''
Computes softmax cross entropy between logits and labels. (deprecated arguments)
'''
def loss_fn(model, images, labels):
  #model 인스턴스 생성
  logits = model(images, training=True)
  #cross entropy 손실함수를 사용해서 손실의 평균값을 구한다.
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
          logits=logits, labels=labels))
  return loss

In [0]:
# calculating gradient
def grad(model, images, labels):
  with tf.GradientTape() as tape:
    loss = loss_fn(model, images, labels)
  # model의 각 variable의 loss의 gradient값을 return한다.
  '''
  print('model variable type: {0}'.format(type(model.variables)))
  print(' ==== model variable list ==== ')
  [tf.keras.models.Sequential.variables]
  https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#variables
  variables:
  Returns the list of all layer variables/weights.

  Alias of self.weights.

  Returns:
  A list of variables.
  '''
  return tape.gradient(loss, model.variables)

In [0]:
# calculating model's accuracy
def evaluate(model, images, labels):
  # model 인스턴스 생성
  logits = model(images, training=True)
  correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  return accuracy

In [0]:
# Enable Eager Mode
tf.enable_eager_execution()

# Hyper Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [0]:
## Dataset 처리
# MNIST Dataset
mnist = keras.datasets.mnist
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

# load mnist dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# image를 0~1 사이의 소수로 변환
train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.
print('shape train, test: {0}, {1}'.format(train_images.shape, test_images.shape))

# 그레이스케일 이미지이므로 채널을 1로 차원을 확장 
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)
print('shape train, test: {0}, {1}'.format(train_images.shape, test_images.shape))

# to_categorical()함수는 숫자를 one-hotcoding 해준다.
print('train_labels[0]: {0}, type: {1}'.format(train_labels[0], type(train_labels[0])))
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)
print('train_labels[0]: {0}, type: {1}'.format(train_labels[0], type(train_labels[0])))

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(
                buffer_size=100000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)
print('datashet shape: {0}'.format(type(train_dataset)))

shape train, test: (60000, 28, 28), (10000, 28, 28)
shape train, test: (60000, 28, 28, 1), (10000, 28, 28, 1)
train_labels[0]: 5, type: <class 'numpy.uint8'>
train_labels[0]: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.], type: <class 'numpy.ndarray'>
datashet shape: <class 'tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter'>


In [0]:
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 2048)              0         
__________

In [0]:
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

In [0]:
# train model
print('Learning started, It takes sometime.')
for epoch in range(training_epochs):
  avg_loss = 0.
  avg_train_acc = 0.
  avg_test_acc = 0.
  train_step = 0
  test_step = 0
  
  # train_dataset의 크기 만큼 반복, 순서대로 image, label쌍을 꺼낸다
  for images, labels in train_dataset:
    # training 용 image와 label의 손실의 gradient를 계산한다.
    grads = grad(model, images, labels)
    optimizer.apply_gradients(zip(grads, model.variables))
    loss = loss_fn(model, images, labels)
    # trainig accuracy를 측정하기 위해 매 횟수마다 총 합을 구한다.
    acc = evaluate(model, images, labels)
    avg_loss = avg_loss+loss
    avg_train_acc = avg_train_acc+acc
    train_step += 1
    
  # 손실, 정확도의 평균을 구한다.
  avg_loss = avg_loss / train_step
  avg_train_acc = avg_train_acc / train_step
  
  # test_dataset의 크기 만큼 반복, 순서대로 image, label쌍을 꺼낸다.
  for images, labels in test_dataset:
    acc = evaluate(model, images, labels)
    avg_test_acc = avg_test_acc + acc
    test_step += 1
  
  # test_dataset의 전체 정확도의 평균을 구한다.
  avg_test_acc = avg_test_acc / test_step
  
  print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))
print('Learning Finished!')

Learning started, It takes sometime.
Epoch: 1 loss = 0.16989940 train accuracy =  0.9476 test accuracy =  0.9829
Epoch: 2 loss = 0.04559535 train accuracy =  0.9860 test accuracy =  0.9824
Epoch: 3 loss = 0.03129787 train accuracy =  0.9898 test accuracy =  0.9872
Epoch: 4 loss = 0.02393986 train accuracy =  0.9926 test accuracy =  0.9884
Epoch: 5 loss = 0.01983924 train accuracy =  0.9941 test accuracy =  0.9904
Epoch: 6 loss = 0.01547315 train accuracy =  0.9951 test accuracy =  0.9889
Epoch: 7 loss = 0.01394321 train accuracy =  0.9955 test accuracy =  0.9877
Epoch: 8 loss = 0.01168457 train accuracy =  0.9966 test accuracy =  0.9911
Epoch: 9 loss = 0.00983523 train accuracy =  0.9969 test accuracy =  0.9899
Epoch: 10 loss = 0.00821953 train accuracy =  0.9975 test accuracy =  0.9894
Epoch: 11 loss = 0.00809129 train accuracy =  0.9973 test accuracy =  0.9918
Epoch: 12 loss = 0.00725020 train accuracy =  0.9976 test accuracy =  0.9898
Epoch: 13 loss = 0.00615880 train accuracy =  0.

 - optimizer.apply_gradient()  
 optimizer.apply_gradient()와 optimizer.minimize()의 차이점에 대한 글  
 https://stackoverflow.com/questions/45473682/difference-between-apply-gradients-and-minimize-of-optimizer-in-tensorflow?rq=1
 
-  optimizer 구현에 대한 더 자세한 내용은 아래 링크를 참고해보자  
 https://www.tensorflow.org/api_docs/python/tf/train/Optimizer
 
- 다른 tutorial들이 여기에 있다. 참고해보자.  
 https://www.tensorflow.org/tutorials